**Chargement des bibliothèques**

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

# Fonction de calcul d'erreur
def RMSE(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
def MAPE(Y_Predicted,Y_actual):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

**Initialisation des données**

In [2]:
# Chargement des données depuis le fichier pkl 
v = pickle.load(open(r'jeu_variables_mmm.pkl','rb'))
entetes = np.array(v['X']).reshape(22,141)[:,0]
X_brut = np.array(v['X']).reshape(22,141)[:,1:141].T.astype('float')
Y_brut = np.array(v['Y']).reshape(140).astype('float')

# Suppression des batteries défectueuses
remove_bat = [8, 10, 12, 13, 22, 96, 117, 126, 136, 137]
X = np.delete(X_brut, remove_bat, 0)
Y = np.delete(Y_brut, remove_bat)

# Suppression des batteries aux durées de vie trop courtes ou trop longues
MIN = 350
MAX = 1350
index_valeurs_faibles = np.where(Y < MIN)
index_valeurs_fortes = np.where(Y > MAX)
index_a_enlever = np.concatenate((index_valeurs_faibles, index_valeurs_fortes), axis=None)
X = np.delete(X, index_a_enlever, 0)
Y = np.delete(Y, index_a_enlever)

# Séparation des jeu d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, test_size=1/3)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, shuffle=True, test_size=1/2, stratify=Y_train)

# Normalisation
def normalize_val(X_train, X_test,):
    sc = StandardScaler(with_mean=True, with_std=True)
    sc.fit(X_train)
    X_train_n = sc.transform(X_train)
    X_test_n = sc.transform(X_test)
    return sc, X_train_n, X_test_n

sc_x,  X_train_n, X_test_n = normalize_val(X_train , X_test)
sc_x_tot, X_n, _ = normalize_val(X, X)

**Dans la partie suivante nous allons entrainer les modèles en séparant nos données en un jeu d'entrainement (2/3) et en un jeu de test (1/3)**

In [3]:
# Modèle Ridge avec Cross Validation
print("Ridge avec Cross Validation avec jeu d'entrainement (2/3) et de test (1/3)\n")
clf_ridge = RidgeCV(alphas=np.logspace(-12,12,20))
clf_ridge.fit(X_train_n, Y_train)

# Prédictions
pred_train = clf_ridge.predict(X_train_n)
pred_test = clf_ridge.predict(X_test_n)    

# Paramètre
print("alpha =",clf_ridge.alpha_,"\n")

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y_train))
print('Train - MAPE=',MAPE(pred_train,Y_train),"%")

# Erreur de test
print('Test - RMSE=',RMSE(pred_test,Y_test))
print('Test - MAPE=',MAPE(pred_test,Y_test),"%")

Ridge avec Cross Validation avec jeu d'entrainement (2/3) et de test (1/3)

alpha = 4.281332398719396 

Train - RMSE= 96.00367205810635
Train - MAPE= 10.948579326197553 %
Test - RMSE= 68.54170062494167
Test - MAPE= 7.443857164457535 %


In [4]:
# Modèle des Random forest
print("Random Forest avec jeu d'entrainement (2/3) et de test (1/3)\n")
clf_random_forest = RandomForestRegressor(max_depth=10)
clf_random_forest.fit(X_train_n, Y_train)

# Prédictions
pred_train = clf_random_forest.predict(X_train_n)
pred_test = clf_random_forest.predict(X_test_n)    

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y_train))
print('Train - MAPE=',MAPE(pred_train,Y_train),"%")

# Erreur de test
print('Test - RMSE=',RMSE(pred_test,Y_test))
print('Test - MAPE=',MAPE(pred_test,Y_test),"%")

Random Forest avec jeu d'entrainement (2/3) et de test (1/3)

Train - RMSE= 52.370528768980364
Train - MAPE= 5.424290084200045 %
Test - RMSE= 101.30256662477477
Test - MAPE= 9.459199991217657 %


In [5]:
# Modèle des processus gaussiens
print("Gaussian Process avec kernel RBF et jeu d'entrainement (2/3) et de test (1/3)\n")
kernel = 1 * RBF(length_scale=1, length_scale_bounds=(1e-3,1e2))
clf_gaussian = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100)
clf_gaussian.fit(X_train_n, Y_train)

# Prédictions
pred_train = clf_gaussian.predict(X_train_n)
pred_test = clf_gaussian.predict(X_test_n)    

# Paramètre
print("Kernel :",clf_gaussian.kernel_,"\n")

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y_train))
print('Train - MAPE=',MAPE(pred_train,Y_train),"%")

# Erreur de test
print('Test - RMSE=',RMSE(pred_test,Y_test))
print('Test - MAPE=',MAPE(pred_test,Y_test),"%")

Gaussian Process avec kernel RBF et jeu d'entrainement (2/3) et de test (1/3)

Kernel : 316**2 * RBF(length_scale=3.78) 

Train - RMSE= 4.857157052203716e-12
Train - MAPE= 3.9031100281285756e-13 %
Test - RMSE= 101.5822753840469
Test - MAPE= 10.230060307607301 %


/home/raphael/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [6]:
# Modèle de régression linéaire
print("Régression linéaire avec jeu d'entrainement (2/3) et de test (1/3)\n")
clf_linear = LinearRegression()
clf_linear.fit(X_train_n, Y_train)

# Prédictions
pred_train = clf_linear.predict(X_train_n)
pred_test = clf_linear.predict(X_test_n)    

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y_train))
print('Train - MAPE=',MAPE(pred_train,Y_train),"%")

# Erreur de test
print('Test - RMSE=',RMSE(pred_test,Y_test))
print('Test - MAPE=',MAPE(pred_test,Y_test),"%")

Régression linéaire avec jeu d'entrainement (2/3) et de test (1/3)

Train - RMSE= 88.92888856861354
Train - MAPE= 10.63805344414866 %
Test - RMSE= 69.92068172525845
Test - MAPE= 8.185679677154154 %


In [7]:
# Modèle LASSO avec Cross Validation
print("LASSO avec Cross Validation avec jeu d'entrainement (2/3) et de test (1/3)\n")
clf_lasso = LassoCV(alphas=np.logspace(-5,5,10), max_iter=100000)
clf_lasso.fit(X_train_n, Y_train)

# Prédictions
pred_train = clf_lasso.predict(X_train_n)
pred_test = clf_lasso.predict(X_test_n)    

# Paramètres
print("Alpha :",clf_lasso.alpha_)
print("Nombre d'itération :",clf_lasso.n_iter_,"\n")

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y_train))
print('Train - MAPE=',MAPE(pred_train,Y_train),"%")

# Erreur de test
print('Test - RMSE=',RMSE(pred_test,Y_test))
print('Test - MAPE=',MAPE(pred_test,Y_test),"%")

LASSO avec Cross Validation avec jeu d'entrainement (2/3) et de test (1/3)

Alpha : 3.593813663804626
Nombre d'itération : 97 

Train - RMSE= 99.22714982140843
Train - MAPE= 11.334257516547416 %
Test - RMSE= 68.50846457769704
Test - MAPE= 7.618568270969107 %


**Dans la partie suivante nous allons entrainer les mêmes modèles avec toutes les données sans utiliser de jeu de test**

In [8]:
# Modèle Ridge appliqué à l'ensemble des variables sans jeu de test
print("Ridge sur l'ensemble des données\n")
clf_ridge_triche = Ridge(alpha=clf_ridge.alpha_)
clf_ridge_triche.fit(X_n, Y)

# Prédiction
pred_train = clf_ridge_triche.predict(X_n)

# Paramètre
print("Alpha :",clf_ridge.alpha_,"\n")

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y))
print('Train - MAPE=',MAPE(pred_train,Y),"%")

Ridge sur l'ensemble des données

Alpha : 4.281332398719396 

Train - RMSE= 84.76484698651021
Train - MAPE= 9.508658554161247 %


In [9]:
# Modèle LASSO appliqué à l'ensemble des variables sans jeu de test
print("LASSO sur l'ensemble des données\n")
clf_lasso_triche = Lasso(alpha=clf_lasso.alpha_ , max_iter=100000)
clf_lasso_triche.fit(X_n, Y)

# Prédiction
pred_train = clf_lasso_triche.predict(X_n)

# Paramètre
print("Nombre d'itération :",clf_lasso_triche.n_iter_)
print("Alpha :",clf_lasso.alpha_,"\n")

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y))
print('Train - MAPE=',MAPE(pred_train,Y),"%")

LASSO sur l'ensemble des données

Nombre d'itération : 100
Alpha : 3.593813663804626 

Train - RMSE= 90.51047511477056
Train - MAPE= 10.138116639522043 %


In [10]:
# Modèle des processus gaussiens appliqué à l'ensemble des variables sans jeu de test
print("Gaussian Process sur l'ensemble des données\n")
clf_gaussian_triche = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100)
clf_gaussian_triche.fit(X_n, Y)

# Prédiction
pred_train = clf_gaussian_triche.predict(X_n)

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y))
print('Train - MAPE=',MAPE(pred_train,Y),"%")

Gaussian Process sur l'ensemble des données

Train - RMSE= 4.819513474898445e-12
Train - MAPE= 4.783196690095543e-13 %


/home/raphael/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [11]:
# Modèle de régression linéaire appliqué à l'ensemble des variables sans jeu de test
print("Régression linéaire sur l'ensemble des données\n")
clf_linear_triche = LinearRegression()
clf_linear_triche.fit(X_n, Y)

# Prédiction
pred_train = clf_linear_triche.predict(X_n)

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y))
print('Train - MAPE=',MAPE(pred_train,Y),"%")

Régression linéaire sur l'ensemble des données

Train - RMSE= 80.46505239530222
Train - MAPE= 9.211511596773324 %


In [12]:
# Modèle des random forest appliqué à l'ensemble des variables sans jeu de test
print("Random Forest sur l'ensemble des données\n")
clf_random_forest_triche = RandomForestRegressor(max_depth=10)
clf_random_forest_triche.fit(X_n, Y)

# Prédiction
pred_train = clf_random_forest_triche.predict(X_n)

# Erreur d'entrainement
print('Train - RMSE=',RMSE(pred_train,Y))
print('Train - MAPE=',MAPE(pred_train,Y),"%")

Random Forest sur l'ensemble des données

Train - RMSE= 39.24816814246546
Train - MAPE= 3.937498902185573 %
